In [2]:
import json
import requests
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.dates as mdates


In [13]:
mcdw = pd.read_csv('../data/mcdw.20230530.csv')
mcdw.info()

C:\Users\Aaron\AppData\Local\Temp\ipykernel_32756\2137799343.py:1: DtypeWarning: Columns (9,13,19) have mixed types. Specify dtype option on import or set low_memory=False.
  mcdw = pd.read_csv('../data/mcdw.20230530.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 907855 entries, 0 to 907854
Data columns (total 36 columns):
 #   Column                                             Non-Null Count   Dtype  
---  ------                                             --------------   -----  
 0   WMO ID                                             907855 non-null  int64  
 1   YEAR-MONTH                                         907855 non-null  object 
 2   LONGITUDE (deg)                                    907855 non-null  float64
 3   LATITUDE (deg)                                     907855 non-null  float64
 4   ELEVATION (m)                                      907855 non-null  float64
 5   STATION PRESSURE (0.1 mb)                          907855 non-null  int64  
 6   STATION PRESSURE SOURCE FLAG                       826655 non-null  object 
 7   MEAN SEA LEVEL PRESSURE (0.1 mb)                   825674 non-null  float64
 8   MEAN SEA LEVEL PRESSURE SOURCE FLAG                907855 non-null  object

In [15]:
weather = mcdw.drop(mcdw.columns[[4, 5, 6,7,8,9,10,11,12,13]], axis=1, inplace=True)